In [1]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1

def parse_events(thisgame_df):
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['play'] = thisgame_df['event'].str.split('/').str.get(0)
    thisgame_df['modifiers'] = thisgame_df['event'].str.split('/',n=1).str.get(1)
    thisgame_df['modifiers'] = thisgame_df['modifiers'].fillna('')
    thisgame_df['baserunning'] = thisgame_df['modifiers'].str.split('.',n=1).str.get(1)
    thisgame_df['baserunning'] = thisgame_df['baserunning'].fillna('')
    thisgame_df['modifiers'] = thisgame_df['modifiers'].str.split('.').str.get(0)
    
    return thisgame_df

In [2]:
# runs in about 5 minutes
with open('events.json', 'r') as f:
#with open('events_2012-2016.json','r') as f:
#with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
allgames_df.head(10)

ok


data  \
id                                                                                                         
BSN192104130  [[cadol101, 4], [mamaa101, 0], [smits102, 0], [oescj101, 4]]                                 
BSN192104140  [[ruetd101, 6], [miljj101, 1], [mitcc102, 3], [mcquh101, 4]]                                 
BSN192104150  [[smits102, 5], [mamaa101, 0], [miljj101, 1], [scotj101, 5]]                                 
BSN192104160  [[cadol101, 5], [mamaa101, 0], [watsm103, 1]]                                                
BSN192104190  [[dougp101, 0], [mcquh101, 4], [scotj101, 0], [coonj101, 1]]                                 
BSN192104200  [[nehfa101, 5], [barnj102, 0], [scotj101, 2], [filld101, 0], [piera101, 1], [towni101, 3]]   
BSN192105040  [[meadl101, 4], [keenj103, 0], [filld101, 2]]                                                
BSN192105070  [[ringj101, 2], [mcquh101, 0]]                                                               
BSN192105100  [[coopw101, 2], [oescj101, 2], [scotj101, 3]]                                                
BSN192105110  [[hamie101, 1], [filld101, 0]]                                                               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
#allgames_df.sample(10).index
#mygames = ['NYN201204100', 'BAL201208060', 'SEA201208150', 'BAL201208300', 'BAL201209300', 'BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

#somecolumns = allgames_df.columns
#allgames2_df = pandas.DataFrame(columns=somecolumns)
#allgames2_df['id'] = ''
#allgames2_df = allgames2_df.set_index('id')
#allgames2_df
#for thisid, thisrow in allgames_df.iterrows():
#    if (thisid in mygames):
#        allgames2_df = allgames2_df.append(thisrow)
#allgames_df = allgames2_df
#print('done')

In [3]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']

allevents_df = pandas.DataFrame(columns = column_list)

cnt = 0


for thisgameID,row in allgames_df.iterrows():
    if (numpy.mod(cnt,100) == 0):
        print('Parsing game {0:,.0f}: {1}'.format(cnt, thisgameID))
    thisgame_df = pandas.DataFrame(row['events'], columns=column_list)
#    thisgame_df = thisgame_df.set_index('event_in_game')

    thisgame_df['gameID'] = thisgameID
    
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df = parse_events(thisgame_df)

    allevents_df = pandas.concat((allevents_df, thisgame_df))
    
    cnt = cnt + 1

allevents_df = allevents_df.reset_index(drop=True)
allevents_df.index.name = 'eventID'

write_columns = ['gameID']
write_columns += ['event_in_game', 'event_type', 'inning', 'home_or_visitor']
write_columns += ['batterID', 'pitch_count', 'pitch_sequence', 'event']
write_columns += ['play', 'modifiers', 'baserunning']
write_columns += ['comment', 'sub_batting_order', 'sub_position']

allevents_df = allevents_df[write_columns]

allevents_df.to_csv('events_parsed_me.csv', encoding='utf-8')
print('Done')
#allevents_df

Parsing game 0: ANA201204060
Parsing game 100: ARI201205210
Parsing game 200: ATL201207020
Parsing game 300: BAL201208120
Parsing game 400: BOS201209210
Parsing game 500: CHN201205050
Parsing game 600: CIN201206220
Parsing game 700: CLE201208060
Parsing game 800: COL201209100
Parsing game 900: HOU201204300
Parsing game 1,000: KCA201206120
Parsing game 1,100: LAN201207170
Parsing game 1,200: MIA201209040
Parsing game 1,300: NYA201204170
Parsing game 1,400: NYN201205270
Parsing game 1,500: OAK201207070
Parsing game 1,600: PHI201208220
Parsing game 1,700: PIT201210030
Parsing game 1,800: SEA201205240
Parsing game 1,900: SFN201206270
Parsing game 2,000: TBA201208070
Parsing game 2,100: TEX201209250
Parsing game 2,200: WAS201205040
Parsing game 2,300: ANA201306050
Parsing game 2,400: ARI201307260
Parsing game 2,500: ATL201309040
Parsing game 2,600: BOS201304212
Parsing game 2,700: CHA201306090
Parsing game 2,800: CHN201307130
Parsing game 2,900: CIN201309020
Parsing game 3,000: COL201304161

KeyboardInterrupt: 